# Split1
In this jupyter notebook we create the schema for the first split of the neural network. This network will have the input and the first hidden layer. After creating the schema we will load the weights corresponding to the first hidden layer to it. 
After loading the weigths to the model we will export it to an unquantized version of tensorflowlite that will run in the first ESP32. 

## Mounting drive

In [ ]:
# Load the Drive helper and mount
from google.colab import drive

# This will prompt for authorization.
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import os 
os.chdir("/content/drive/MyDrive/spiltNN/hello_world_esp32_split/")
!ls

converted_model.tflite	   hello_world_joined_split.ipynb  models
esp_split1		   hello_world_model.h5		   output_split.pickle
esp_split2		   hello_world_split1.ipynb	   x_test.pickle
hello_world_full_NN.ipynb  hello_world_split2.ipynb


In [ ]:
# TensorFlow is an open source machine learning library
import tensorflow as tf

# Keras is TensorFlow's high-level API for deep learning
from tensorflow import keras
# Numpy is a math library
import numpy as np
# Pandas is a data manipulation library 
import pandas as pd
# Matplotlib is a graphing library
import matplotlib.pyplot as plt
# Math is Python's math library
import math

# Set seed for experiment reproducibility
seed = 1
np.random.seed(seed)
tf.random.set_seed(seed)

# Create model schema
We create the model schema for the first split of the neural network. This is the first layer that has 16 neurons an a relu activation function. 
For the model to be updated correctly the name of the layer in this schema must be the same as the one assigned to the layer in the full hello_world model. In this case the first hidden layer has the name `first_layer` in both models. 

In [ ]:
split1_model = tf.keras.Sequential()

# First layer takes a scalar input and feeds it through 16 "neurons". The
# neurons decide whether to activate based on the 'relu' activation function.
split1_model.add(keras.layers.Dense(16, activation='relu', name="first_layer", input_shape=(1,)))


## Upload weights
We upload the weights to the model we just created and check that the model parameters coincide with the ones of the full neural network. The parameter `by_name=True` ensures that the weights are uploaded to each layer properly. 

In [ ]:
split1_model.load_weights("hello_world_model.h5", by_name=True)
split1_model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 first_layer (Dense)         (None, 16)                32        
                                                                 
Total params: 32
Trainable params: 32
Non-trainable params: 0
_________________________________________________________________


## Export model to tflite model 
We quantize the model and then export it to tensoflow lite. The resulting file is a `.tflite` file, this file can be exported to a c++ file that contains the weights of the network. This file can be either quantized or not quantized
### Quantized


In [ ]:
converter = tf.lite.TFLiteConverter.from_keras_model(split1_model)
converter.optimizations = [tf.lite.Optimize.DEFAULT]
def representative_dataset_gen():
    for _ in range(10000):
        yield [
            np.array(
                [np.random.uniform(), np.random.uniform()]
            , dtype=np.float32)
        ]
converter.representative_dataset = representative_dataset_gen
converter.target_spec.supported_ops = [tf.lite.OpsSet.TFLITE_BUILTINS_INT8]
tflite_quant_model = converter.convert()
open("converted_model_split1.tflite", "wb").write(tflite_quant_model)

INFO:tensorflow:Assets written to: /tmp/tmp_tbtmb1l/assets


/usr/local/lib/python3.7/dist-packages/tensorflow/lite/python/convert.py:746: UserWarning: Statistics for quantized inputs were expected, but not specified; continuing anyway.
  warnings.warn("Statistics for quantized inputs were expected, but not "


1480

### No quantized


In [ ]:
# Convert the model to the TensorFlow Lite format without quantization
converter = tf.lite.TFLiteConverter.from_keras_model(split1_model)
model_no_quant_tflite = converter.convert()

# Save the model to disk
open("converted_model_split1_noquant.tflite", "wb").write(model_no_quant_tflite)

# Convert the model to the TensorFlow Lite format with quantization
def representative_dataset():
  for i in range(500):
    yield([x_train[i].reshape(1, 1)])
# Set the optimization flag.
converter.optimizations = [tf.lite.Optimize.DEFAULT]
# Enforce integer only quantization
converter.target_spec.supported_ops = [tf.lite.OpsSet.TFLITE_BUILTINS_INT8]
converter.inference_input_type = tf.int8
converter.inference_output_type = tf.int8
# Provide a representative dataset to ensure we quantize correctly.
converter.representative_dataset = representative_dataset
model_tflite = converter.convert()

# Save the model to disk
open("converted_model_split1.tflite", "wb").write(model_tflite)

INFO:tensorflow:Assets written to: /tmp/tmpam2gcdhp/assets


INFO:tensorflow:Assets written to: /tmp/tmpam2gcdhp/assets


INFO:tensorflow:Assets written to: /tmp/tmp3qgwhmfe/assets


INFO:tensorflow:Assets written to: /tmp/tmp3qgwhmfe/assets
/usr/local/lib/python3.7/dist-packages/tensorflow/lite/python/convert.py:746: UserWarning: Statistics for quantized inputs were expected, but not specified; continuing anyway.
  warnings.warn("Statistics for quantized inputs were expected, but not "


1152

## To convert to C++
We can then run this command to convert the model to c code.
```
xxd -i converted_model.tflite > model_data.cc
```